In [1]:
import pandas as pd
import numpy as np

In [2]:
path='/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/'

# filtrowanie pliku wynikowego blasta

In [3]:
df_blast_results = pd.read_csv(path+'novel_counted_1d-6d-9d_salmo_blasted.csv', sep='\t', low_memory=False)
df_blast_results.shape

(4271458, 5)

### odfiltrowanie zbyt niskiego coverage (pomijamy w tej wersji salmo only)

### filtrowanie mRNA only

In [4]:
# select best hit based on evalue (drop_duplicates)
df_best_hit = df_blast_results.sort_values('evalue', ascending=True).drop_duplicates('query').sort_index()


In [5]:
df_best_hit.shape

(104024, 5)

In [6]:
# select what you need based on string

# w tym wypadku mRNA only oznacza pominiecie blastów z ncRNA, mixRNA i innych dziwnych, nie oznacza natomiast
# zabawy z chromosome i filtrowaniem tego dodatkowo, poniewaz blastujac tylko do salmo salar, nie mam (chyba)
# zadnych hitow chromosome. Chromosome (czyli hity bez mRNA) w tym wypadku to takie, ktore w blascie mialy NO HIT.
# czyli zamiast filtrowania po chromosome nalezy wrzucic w te grupe trinity, ktore mialy NO HIT.

df_mRNA_only = df_best_hit[df_best_hit['al_title'].str.contains('mRNA')]
df_mRNA_only.shape


(93064, 5)

In [7]:
df_mRNA_only.to_csv(path+'PBS_vs_MC_all_days_mRNA_only.csv', index=False)

In [8]:
mRNA_XM = df_mRNA_only[['al_accession']]#.to_csv(path+'xm_list_to_convet_to_xp.txt', index=False)


In [9]:
mRNA_XM.shape

(93064, 1)

In [10]:
mRNA_XM.drop_duplicates(inplace=True)
mRNA_XM.shape

/home/maciek/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


(30010, 1)

# TO DO: XM z tego ^^ pozamieniac na XP w unirefie

In [ ]:
not mapped
XR_001330223
XR_001324603
NM_001360919
XR_001324726
NM_001361122
NM_001360942


30004 out of 30010 RefSeq Nucleotide identifiers were successfully mapped to 27345 UniProtKB IDs in the table below. 


In [62]:
uniprot_converted = pd.read_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/xm_to_uniprot.tab', sep='\t')

In [49]:
uniprot_converted.head(1)

,Entry,Protein names,Gene names,al_accession
0,A0A1S3P3U9,forkhead box protein P4-like isoform X8,LOC106583051,XM_014166828


In [63]:
uniprot_converted.shape

(27345, 4)

In [70]:
# w przypadku jesli dany hit ma dwa lub wiecej numerow accession (XM_014212336,XM_014212334)
# rozdzielenie na osobne wiersze, z ktorych kazdy bedzie mial teraz tylko jeden numer
#new_df = pd.DataFrame(df.City.str.split('|').tolist(), 

uniprot_converted_split_al_accession = pd.DataFrame(uniprot_converted['al_accession'].str.split(',').tolist(), index=uniprot_converted.Entry).stack()
uniprot_converted_split_al_accession = uniprot_converted_split_al_accession.reset_index([0, 'Entry'])
#uniprot_converted_split_al_accession.to_csv("/home/maciek/Pobrane/test.csv")
#uniprot_converted_split_al_accession.reset_index()
uniprot_converted_split_al_accession.columns = ['Entry', 'al_accession']
uniprot_converted_split_al_accession.head()

,Entry,al_accession
0,A0A1S3P3U9,XM_014166828
1,A0A1S3T2R9,XM_014215408
2,A0A1S3SRF3,XM_014211429
3,A0A1S3RP07,XM_014198580
4,A0A1S3NG44,XM_014158943


In [76]:
# merge with other columns from uniprot_converted

uniprot_converted_split_al_accession_w_protein_gene_names = pd.merge(uniprot_converted, uniprot_converted_split_al_accession, on='Entry')
del uniprot_converted_split_al_accession_w_protein_gene_names['al_accession_x']
uniprot_converted_split_al_accession_w_protein_gene_names.rename(columns={'al_accession_y':'al_accession'}, inplace=True)
uniprot_converted_split_al_accession_w_protein_gene_names.head()


,Entry,Protein names,Gene names,al_accession
0,A0A1S3P3U9,forkhead box protein P4-like isoform X8,LOC106583051,XM_014166828
1,A0A1S3T2R9,protein CASP-like,LOC106613315,XM_014215408
2,A0A1S3SRF3,DNA (Cytosine-5)-methyltransferase 3A-like iso...,LOC106611330,XM_014211429
3,A0A1S3RP07,uncharacterized protein LOC106604161,LOC106604161,XM_014198580
4,A0A1S3NG44,serine/arginine repetitive matrix protein 1-li...,LOC106579139,XM_014158943


In [41]:
#uniprot_converted_entry = uniprot_converted[['al_accession', 'Entry']]

In [77]:
merged = pd.merge(df_mRNA_only, uniprot_converted_split_al_accession_w_protein_gene_names, on='al_accession', how='inner') 

In [78]:
merged.shape

(93208, 8)

In [86]:
merged.head()

,query,al_title,al_accession,evalue,coverage,Entry,Protein names,Gene names
0,TRINITY_DN100004_c4_g2_i2,gi|929290304|ref|XM_014166828.1| PREDICTED: Sa...,XM_014166828,0.000000e+00,100.183824,A0A1S3P3U9,forkhead box protein P4-like isoform X8,LOC106583051
1,TRINITY_DN100005_c7_g2_i1,gi|929057222|ref|XM_014212676.1| PREDICTED: Sa...,XM_014212676,0.000000e+00,83.712322,A0A1S3SUX9,"alpha-N-acetylgalactosaminide alpha-2,6-sialyl...",LOC106611954
2,TRINITY_DN81030_c8_g4_i1,gi|929057222|ref|XM_014212676.1| PREDICTED: Sa...,XM_014212676,5.808660e-77,100.000000,A0A1S3SUX9,"alpha-N-acetylgalactosaminide alpha-2,6-sialyl...",LOC106611954
3,TRINITY_DN83202_c8_g4_i2,gi|929057222|ref|XM_014212676.1| PREDICTED: Sa...,XM_014212676,2.955540e-50,44.669366,A0A1S3SUX9,"alpha-N-acetylgalactosaminide alpha-2,6-sialyl...",LOC106611954
4,TRINITY_DN105298_c5_g1_i2,gi|929057222|ref|XM_014212676.1| PREDICTED: Sa...,XM_014212676,3.652550e-55,93.772894,A0A1S3SUX9,"alpha-N-acetylgalactosaminide alpha-2,6-sialyl...",LOC106611954


In [ ]:
# i chyba na tym powinienem poprzestac. Mam entry dla kazdego trinity (query) i teraz
# polacze to z matryca UTR (trinity+entry)
# to samo dla listy bialek i polaczyc obie matryce utr i prot w jedna matryce

In [81]:
merged.to_csv(path+'mRNA_only_merged_w_XP.csv', index=False)
#merged.to_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/6d/salmo_salar_only/FINAL_RESULT/merged.csv', index=False)
#merged.to_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/9d/salmo_salar_only/FINAL_RESULT/merged.csv', index=False)

In [79]:
# sprawdzenie czy merge zostal zrobiony dobrze: czy te, ktore nie pojawily sie w merged rzeczywiscie nie zostaly zmapowane

test_if_merged_correctly = df_mRNA_only[~df_mRNA_only['query'].isin(merged['query'])]
test_if_merged_correctly.shape

(16, 5)

In [80]:
test_if_merged_correctly # wyglada ze wszystko jest juz ok, zostaly tylko nie zmapowane z listy powyzej.

,query,al_title,al_accession,evalue,coverage
15271,TRINITY_DN100586_c3_g3_i1,gi|929113998|ref|XR_001330223.1| PREDICTED: Sa...,XR_001330223,0.000000e+00,70.874751
190183,TRINITY_DN107559_c0_g1_i1,gi|929309491|ref|XR_001324603.1| PREDICTED: Sa...,XR_001324603,7.882420e-57,90.510949
562362,TRINITY_DN84073_c2_g3_i1,gi|1344537309|ref|NM_001360919.1| Salmo salar ...,NM_001360919,2.131430e-58,66.568047
933439,TRINITY_DN96039_c7_g2_i1,gi|929113998|ref|XR_001330223.1| PREDICTED: Sa...,XR_001330223,1.553100e-09,29.000000
1057874,TRINITY_DN101445_c8_g5_i1,gi|1344537309|ref|NM_001360919.1| Salmo salar ...,NM_001360919,1.253290e-37,23.900574
2328356,TRINITY_DN100513_c3_g1_i2,gi|929113998|ref|XR_001330223.1| PREDICTED: Sa...,XR_001330223,1.151990e-51,32.558140
2503615,TRINITY_DN113439_c10_g5_i2,gi|1344537309|ref|NM_001360919.1| Salmo salar ...,NM_001360919,3.068830e-60,81.702128
2634549,TRINITY_DN89745_c6_g1_i1,gi|1344537309|ref|NM_001360919.1| Salmo salar ...,NM_001360919,1.229830e-38,67.164179
2827679,TRINITY_DN109765_c5_g1_i2,gi|929312198|ref|XR_001324726.1| PREDICTED: Sa...,XR_001324726,0.000000e+00,101.380898
3053488,TRINITY_DN83351_c12_g1_i2,gi|1344537309|ref|NM_001360919.1| Salmo salar ...,NM_001360919,1.566220e-46,39.181287


krok 2: na podsawie 'Entry' polaczyc z tabela bialek

In [82]:
# uniprot_proteins_all to wszystkie bialka, jakie udalo sie zblastowac trinity.
uniprot_proteins_all = pd.read_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/proteins_salmo/uniprot_all_proteins.tab', sep='\t')

In [83]:
mRNA_with_proteins = pd.merge(merged, uniprot_proteins_all, on='Entry', suffixes=('_UTR', '_PROTEIN'))

In [84]:
mRNA_with_proteins.shape

(55322, 16)

In [85]:
mRNA_with_proteins.to_csv(path+'mRNA_UTR_PROTEINS_merged.csv', index=False)
#merged_with_proteins_deseq.to_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/6d/salmo_salar_only/FINAL_RESULT/mRNA_UTR_PROTEINS_DESEQ_merged_6d.csv', index=False)
#merged_with_proteins_deseq.to_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/9d/salmo_salar_only/FINAL_RESULT/mRNA_UTR_PROTEINS_DESEQ_merged_9d.csv', index=False)

### ponizsze nie wiem czy bedzie potrzebne, zostawiam na wszelki wypadek

In [16]:
# łączenie odfiltrowanych mRNA z wynikami deseq

df_mRNA_only.rename(columns={'query' : 'ID'}, inplace=True)
df_mRNA_only.head()

/home/maciek/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,ID,al_title,al_accession,evalue,coverage
0,TRINITY_DN100004_c4_g2_i2,gi|929290304|ref|XM_014166828.1| PREDICTED: Sa...,XM_014166828,0.000000e+00,100.183824
12,TRINITY_DN100005_c7_g2_i1,gi|929057222|ref|XM_014212676.1| PREDICTED: Sa...,XM_014212676,0.000000e+00,83.712322
58,TRINITY_DN100005_c8_g3_i1,gi|929100757|ref|XM_014205961.1| PREDICTED: Sa...,XM_014205961,2.836380e-18,8.125000
75,TRINITY_DN100005_c9_g1_i1,gi|929082593|ref|XM_014196305.1| PREDICTED: Sa...,XM_014196305,7.129640e-39,51.275510
87,TRINITY_DN100008_c3_g3_i1,gi|929118624|ref|XM_014215408.1| PREDICTED: Sa...,XM_014215408,8.892420e-18,17.934783


In [17]:
result = pd.merge(df_deseq_out, df_mRNA_only, on='ID')
#result.sort_values(by='log2FoldChange', inplace=True)

In [25]:
result.shape

(74221, 17)

In [21]:
result.to_csv(path+'FINAL_RESULT/mRNA_PBS_1d_vs_MC2_1d_deseq_w_mRNA_names.csv', index=False)

In [24]:
result_pajd = result[result['padj'] < 0.001]

In [26]:
result_pajd.shape

(3605, 17)

### filtrowanie hitów innych niz mRNA: ncRNA, miscRNA. ZDECYDOWAC CO Z NIMI

In [28]:
non_mRNA = df_best_hit[~df_best_hit['al_title'].str.contains('mRNA')]


In [29]:

non_mRNA.shape

(6, 5)

In [30]:
non_mRNA.head(1)

,query,al_title,al_accession,evalue,coverage
1045,TRINITY_DN106385_c5_g2_i1,gi|929246779|ref|XR_001320874.1| PREDICTED: Sa...,XR_001320874,4.447370e-19,15.862069


In [31]:
non_mRNA.rename(columns={'query' : 'ID'}, inplace=True)

In [32]:
result2 = pd.merge(df_deseq_out, non_mRNA, on='ID')
result2.sort_values(by='log2FoldChange', inplace=True)

In [33]:
result2.to_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/9d/salmo_salar_only/FINAL_RESULT/non_mRNA_PBS_1d_vs_MC2_9d_deseq_w_mRNA_names.csv', index=False)

### filtrowanie NO HIT
#### chromosome poprzednio == no hit teraz

In [12]:
# wszystkie blastowane trinity
all_trinity_to_blast = pd.read_csv(path+'novel_counted-1d-6d-9d.headers')

In [13]:
all_trinity_to_blast.shape

(188998, 1)

In [14]:
all_trinity_to_blast.head()

,id
0,TRINITY_DN100003_c1_g1_i1
1,TRINITY_DN100004_c4_g2_i2
2,TRINITY_DN100005_c7_g2_i1
3,TRINITY_DN100005_c8_g3_i1
4,TRINITY_DN100005_c9_g1_i1


In [15]:
df_best_hit.rename(columns={'query' : 'ID'}, inplace=True)

In [16]:
# wszystkie odjac te ktore mialy hit w blascie, jakikolwiek hit
no_hit = all_trinity_to_blast[~all_trinity_to_blast['id'].isin(df_best_hit['ID'])]

In [17]:
no_hit.shape

(84974, 1)

In [19]:
no_hit.to_csv(path+'NO_HIT_84974.headers', index=False)

In [40]:
no_hit.rename(columns={'id' : 'ID'}, inplace=True)

/home/maciek/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [41]:
# RACZEJ BREDNIE STARE
# lista NO HIT czyli dawna chromosomes, ktora idzie do analizy novel lncRNA
#result3 = pd.merge(df_deseq_out, no_hit, on='ID')
#result3.sort_values(by='log2FoldChange', inplace=True)

In [42]:
#result3.to_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/9d/salmo_salar_only/FINAL_RESULT/NO_HIT_PBS_1d_vs_MC2_9d_deseq_w_mRNA_names.csv', index=False)

In [18]:
wszystkie_test = 104024+84974 # jakis hit + no hit
wszystkie_test

188998